In [1]:
# imports and loading
import pandas as pd
import pickle
import numpy as np

# load in settings flags
flags_filepath = '../options/gene_flags/new_gene_flags.csv'
TF_flags_filepath = '../options/TF_flags/new_TF_flags.csv'

# generate gene flags options template

In [ ]:
# loading

# load in a bunch of potentially useful files
# loading merged log_tpm_file
# merge together log_tpm_df files
log_tpm_df = pd.read_csv('../data/external/imodulon_info/corrected/PRECISE_1K_log_tpm_basal.csv', index_col = 0)
starve_log_tpm = pd.read_csv('../data/external/validation_data_sets/stationary_phase/cleaned_log_tpm_qc.csv', index_col = 0)
to_blank_inds = list(set(log_tpm_df.index) - set(starve_log_tpm.index))
# need to create zero rows for missing values
zeros_data = {col : 0 for col in starve_log_tpm.columns}
zeros_df = pd.DataFrame(zeros_data, index = to_blank_inds)
starve_log_tpm = pd.concat([starve_log_tpm, zeros_df])
starve_log_tpm = starve_log_tpm.loc[log_tpm_df.index]
log_tpm_df = pd.concat([starve_log_tpm, log_tpm_df], axis = 1)
precise_log_tpm_df = pd.read_csv('../data/external/imodulon_info/corrected/PRECISE_1K_log_tpm_basal.csv', index_col = 0)

iM_to_genes = {}
gene_to_iMs = {}
k_to_iM = {}
f = open('../data/external/imodulon_info/iM_table.csv', 'r')
lines = f.readlines()
f.close()
k_to_iM = {line.split(',')[0] : line.split(',')[1] for line in lines[1:]}
iM_to_regulator = {line.split(',')[1] : line.split(',')[2] for line in lines[1:]}
f = open('../data/external/imodulon_info/corrected/gene_presence_matrix.csv', 'r')
lines = f.readlines()
f.close()

gene_presence_df = pd.read_csv('../data/external/imodulon_info/corrected/gene_presence_matrix.csv', index_col = 0)
gene_presence_df = gene_presence_df.rename(columns = k_to_iM)
for gene in gene_presence_df.index:
    iMs_gene_in = gene_presence_df.columns[gene_presence_df.loc[gene] == True].to_list()
    for iM in iMs_gene_in:
        if iM not in iM_to_genes:
            iM_to_genes.update({iM : []})
        iM_to_genes[iM].append(gene)
        if gene not in gene_to_iMs:
            gene_to_iMs.update({gene : []})
        gene_to_iMs[gene].append(iM)
    
A = pd.read_csv('../data/external/imodulon_info/corrected/PRECISE_1K_A_basal.csv', index_col = 0).rename(index = {int(k) : v for k, v in k_to_iM.items()})
M = pd.read_csv('../data/external/imodulon_info/corrected/PRECISE_1K_M_basal.csv', index_col = 0).rename(columns = k_to_iM)

TRN_df = pd.read_pickle('../data/external/regulonDB_files/TRN_df.pkl') # TRN_df.loc[regulator][target] = 1 means regulator regulates target

pickle_in = open('../data/interim/misc_dictionaries/b_to_gene.pkl', 'rb')
b_to_gene = pickle.load(pickle_in)
pickle_in.close()
gene_to_b = {v : k for k, v in b_to_gene.items()}
gene_to_b.update({'ntrC' : 'b3868'})
iM_to_b_regulator = {
    'GlpR' : ['b3423'], 
    'DhaR/Mlc' : ['b1201', 'b1594'], 
    'ArgR' : ['b3237'], 
    'NarL' : ['b1221'], 
    'MalT' : ['b3418'], 
    'OxyR' : ['b3961'], 
    'Crp-2' : ['b3357'], 
    'AtoC' : ['b2220'], 
    'PurR-1' : ['b1658'], 
    'CsqR' : ['b3884'], 
    'MetJ' : ['b3938'], 
    'PurR-2' : ['b1658'], 
    'Cbl+CysB' : ['b1987' 'b1275'], 
    'GlcC' : ['b2980'], 
    'Crp-1' : ['b3357'], 
    'YiaJ' : ['b3574'], 
    'GcvA' : ['b2808'], 
    'XylR' : ['b3569'], 
    'RpoH' : ['b3461'], 
    'GadEWX' : ['b3512', 'b3515', 'b3516'], 
    'PrpR' : ['b0330'], 
    'GadWX' : ['b3516', 'b3515'], 
    'CecR' : ['b0796'], 
    'ArcA-1' : ['b4401'], 
    'Fnr' : ['b1334'], 
    'CdaR' : ['b0162'], 
    'Pyruvate' : ['b2125', 'b2381', 'b0113'], 
    'SrlR+GutM' : ['b2707', 'b2706'], 
    'NagC/TyrR' : ['b0676', 'b1323'], 
    'CpxR' : ['b3912'], 
    'Lrp' : ['b0889'], 
    'Leu/Ile' : ['b3773'], 
    'Fur-1' : ['b0683'], 
    'CysB' : ['b1275'], 
    'FliA' : ['b1922'], 
    'PuuR' : ['b1299'], 
    'NikR' : ['b3481'], 
    'NtrC+RpoN' : ['b3868', 'b3202'], 
    'RcsAB' : ['b1951', 'b2217'], 
    'Fur-2' : ['b0683'], 
    'Copper' : ['b0571', 'b1969', 'b0487'], 
    'AllR/AraC/FucR' : ['b0506', 'b0064', 'b2805'], 
    'Cra' : ['b0080'], 
    'ArcA-2' : ['b4401'], 
    'EvgA' : ['b2369'], 
    'Zinc' : ['b3292', 'b4046'], 
    'SoxS' : ['b4062'], 
    'Nac' : ['b1988'], 
    'Tryptophan' : ['b4393'], 
    'FadR/IclR' : ['b1187', 'b4018'], 
    'FlhDC' : ['b1892', 'b1891'], 
    'GntR/TyrR' : ['b3438', 'b1323'], 
    'RpoS' : ['b2741'], 
    'ExuR/FucR' : ['b3094', 'b2805'], 
    'FecI' : ['b4293'], 
    'RbsR' : ['b3753']
}
iM_to_b_regulator = {}
iM_table = pd.read_csv('../data/external/imodulon_info/iM_table.csv', index_col = 0)
for index, row in iM_table.iterrows():
    iM = row['name']
    regulator = row['regulator_readable']
    if type(regulator) == float or len(regulator) > 6:
        continue
    regulator = regulator[0].lower()+regulator[1:]
    if regulator == 'flhDC':
        regulator = 'flhD'
    if regulator == 'rcsAB':
        regulator = 'rcsA'
    if regulator == 'ppGpp':
        continue
    b = gene_to_b[regulator]
    iM_to_b_regulator.update({iM : [b]})
iM_to_b_regulator.update({'Purine' : ['b1658']})
iM_to_b_regulator.update({'DhaR' : ['b1201', 'b1594']})


# generate mapping information
num_iMs_to_ct = {}
for gene in log_tpm_df.index:
    if gene not in gene_to_iMs:
        num_iMs = 0
    else:
        num_iMs = len(gene_to_iMs[gene])
    if num_iMs not in num_iMs_to_ct:
        num_iMs_to_ct.update({num_iMs : 0})
    num_iMs_to_ct[num_iMs] += 1

# out of genes with iModulons, how many are actually regulated by the iM's regulators?
all_overlap = []
some_overlap = []
no_overlap = []
iMs_have_no_regs = []
for gene, iMs in gene_to_iMs.items():
    if gene not in TRN_df.columns:
        no_overlap.append(gene)
    else:
        regs = TRN_df.index[TRN_df[gene] != 0].to_list()
        overlapped = 0
        iMs_have_regs = 0
        for iM in iMs:
            if iM not in iM_to_b_regulator:
                continue
            iMs_have_regs += 1
            if len(set(iM_to_b_regulator[iM]).intersection(regs)) > 0:
                overlapped += 1
        if iMs_have_regs == 0:
            iMs_have_no_regs.append(gene)
        elif iMs_have_regs == overlapped:
            all_overlap.append(gene)
        elif overlapped == 0:
            no_overlap.append(gene)
        else:
            some_overlap.append(gene)

In [ ]:
# create flags template
defaults = {
    'checked' : False,
    'include' : False,
    'force_rerun': False,
    'force_basal_rerun' : False,
    'sanity_plots': True,
    'grid_use': -7,
    'basal_bool': False,
    'basal_or_hard_val': 'basal',
    'hard_val': 10.0,
    'use_zerod_A_matrix': True,
    'initial_guess_ratio': 0.3,
    'use_target_range': False,
    'target_range': '[-1, 3]',
    'base_cInhibitor_val': 0.2,
    'base_cActivator_val': 0.2,
    'auto_set_max_range': True,
    'additional_tolerance': 0.2,
    'cActivator': '[-4, 2]',
    'cInhibitor': '[-4, 2]',
    'run_greedy': True,
    'neg_grid_toss_OR_zero': 'toss',
    'seed': 42.0,
    'n_ind': 100.0,
    'mu': 100.0,
    'lambda_': 100.0,
    'cxpb': 0.6,
    'cx_prob': 0.6,
    'mutpb': 0.4,
    'mt_prob': 0.1,
    'n_gen': 100.0,
    'verbose': False,
    'n_iter': 5.0,
    'max_steps': 30.0,
    'n_rounds': 100.0
}
cols_order = [
    'checked', 
    'include', 
    'force_rerun', 
    'force_basal_rerun', 
    'sanity_plots', 
    'act_iM', 
    'inh_iM', 
    'basal_conditions', 
    'grid_use', 
    'basal_bool', 
    'basal_or_hard_val', 
    'hard_val', 
    'use_zerod_A_matrix', 
    'initial_guess_ratio', 
    'use_target_range', 
    'target_range', 
    'base_cInhibitor_val', 
    'base_cActivator_val', 
    'auto_set_max_range', 
    'additional_tolerance', 
    'cActivator', 
    'cInhibitor', 
    'run_greedy', 
    'neg_grid_toss_OR_zero', 
    'seed', 
    'n_ind', 
    'mu', 
    'lambda_', 
    'cxpb', 
    'cx_prob', 
    'mutpb', 
    'mt_prob', 
    'n_gen', 
    'verbose', 
    'n_iter', 
    'max_steps', 
    'n_rounds'
]



act_iMs_list = []
inh_iMs_list = []
genes = []
for gene in all_overlap:
    iMs = gene_to_iMs[gene]
    regs = TRN_df.index[TRN_df[gene] != 0].to_list()
    
    # need to figure out if for said gene it is a promoter or inhibitor
    # with Gabe's iModulons, it's just if it's weighting is positive or negative
    act_iMs = []
    inh_iMs = []
    for iM in iMs:
        weighting = M[iM].loc[gene]
        if weighting > 0:
            act_iMs.append(iM)
        else:
            inh_iMs.append(iM)
    if (len(act_iMs) > 1 or len(inh_iMs) > 1) or (len(act_iMs) == 0 and len(inh_iMs) == 0):
        continue
    if len(act_iMs) == 0:
        act_iMs = [float('nan')]
    if len(inh_iMs) == 0:
        inh_iMs = [float('nan')]
    genes.append(gene)
    act_iMs_list.append(act_iMs[0])
    inh_iMs_list.append(inh_iMs[0])
    
new_flags = pd.DataFrame(index = genes, columns = cols_order)
new_flags['act_iM'] = act_iMs_list
new_flags['inh_iM'] = inh_iMs_list
for col, val in defaults.items():
    new_flags[col] = [val for _ in range(len(new_flags.index))]

new_flags.to_csv(flags_filepath)

# generate TF flags options template

In [ ]:
# setup and create
all_iMs = list(set(new_flags.act_iM).union(new_flags.inh_iM))
columns = [
    'TF', 
    'gene_name', 
    'effectors', 
    'filter_outliers_less_than_X_corr_to_rest', 
    'cAct_no_effector_form', 
    'cAct_multi_effector_binding', 
    'cAct_multi_co_effector_binding', 
    'cInh_no_effector_form', 
    'cInh_multi_effector_binding', 
    'cInh_multi_co_effector_binding', 
    'kd_inh_metab_up', 
    'kd_inh_metab_lo', 
    'kd_act_metab_up', 
    'kd_act_metab_lo', 
]
TF_df = pd.DataFrame(index = all_iMs, columns = columns)
TF_df['filter_outliers_less_than_X_corr_to_rest'] = 0.3
TF_df.to_csv(TF_flags_filepath)